构造数据集

In [1]:
import os
import SimpleITK as sitk
import numpy as np
import scipy
import pandas as pd

In [2]:
image_path = r"C:\Users\HongDQ\Desktop\VOI_resample_normal\image"
mask_path = r"C:\Users\HongDQ\Desktop\VOI_resample_normal\mask"
image_list = os.listdir(image_path)
image_list.sort()
mask_list = os.listdir(mask_path)
mask_list.sort()
ID_total = []
image_max_slic_total = []
try:
    for i in range(len(mask_list)):
        ID = image_list[i][0:6]
        image = sitk.ReadImage(image_path + "\\" + image_list[i])
        mask = sitk.ReadImage(mask_path + "\\" + mask_list[i])
        image_array = sitk.GetArrayFromImage(image)
        mask_array = sitk.GetArrayFromImage(mask)
        #裁剪3D立体框
        # tumor_coordinates = np.argwhere(mask_array == 1)
        # bbox = np.array([[min(coord[axis] for coord in tumor_coordinates), max(coord[axis] for coord in tumor_coordinates)] for axis in range(3)])
        # mask_array = mask_array[bbox[0][0]:bbox[0][1],bbox[1][0]:bbox[1][1],bbox[2][0]:bbox[2][1]]
        # image_array = image_array[bbox[0][0]:bbox[0][1],bbox[1][0]:bbox[1][1],bbox[2][0]:bbox[2][1]]
        # zero_mask = np.argwhere(mask_array == 0)
        # image_array[zero_mask[:, 0], zero_mask[:, 1], zero_mask[:, 2]] = 0
        #裁剪最大切片
        area = np.sum(mask_array == 0, axis=(1, 2))
        area_index = np.argsort(area)[-1]
        img = image_array[area_index, :, :]
        output_shape = [224,224]    #resize
        sample = scipy.ndimage.zoom(img, zoom=(output_shape[0]/img.shape[0],output_shape[1]/img.shape[1]), output=None, order=3, mode='constant', cval=0.0, prefilter=True)
        image_max_slic_total.append(sample)
        ID_total.append(ID)
        # image = sitk.GetImageFromArray(sample)
        # name =ID + "tumor.nii.gz"
        # sitk.WriteImage(image,name,True)
except Exception as e:
    print(ID)

deskto


In [ ]:
MRI_array_datasets = torch.Tensor(np.array(image_max_slic_total)[:, np.newaxis, :, :])
label = pd.read_csv("./only_tumor_clinical.csv")
MRI_PCR_label = torch.Tensor(label.values[:,5].astype(int))


In [ ]:
ID_total = pd.DataFrame(ID_total)
ID_total = ID_total.rename(columns={ID_total.columns[0]:'ID'})
ID_total['ID'] = ID_total['ID'].astype(int)
clinical_excel = pd.read_csv("./Clinical imformation.csv")
clinical_excel = clinical_excel.sort_values(by=clinical_excel.columns[0])
total_result = pd.merge(ID_total,clinical_excel,on='ID',how='inner')
label = total_result.values[:,5]
label = np.array(label).astype(int)
# label = torch.from_numpy(label)


In [4]:
import os
import torch
import monai
import numpy as np
import pandas as pda
import torch.nn as nn
import SimpleITK as sitk
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold,train_test_split
from monai.networks.nets.resnet import ResNetBottleneck as Bottleneck

c:\Users\HongDQ\.conda\envs\fb\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MRI_path = r"./only_tumor_MRI_601300_256 3"
MRI_list = os.listdir(MRI_path)
MRI_list.sort()
MRI_array_datasets = []
for i in range(len(MRI_list)):
    image = sitk.ReadImage(MRI_path + "\\" + MRI_list[i])
    image_array = sitk.GetArrayFromImage(image)
    # area = np.sum(image_array == 1, axis=(1, 2))
    # area_index = np.argsort(area)[-1]
    # mask = image_array[area_index, :, :]
    total_sum_per_slice = np.sum(image_array, axis=(0, 1))
    max_total_sum_slice_index = np.argmax(total_sum_per_slice)
    max_total_sum_slice = image_array[:, :, max_total_sum_slice_index]
    MRI_array_datasets.append(max_total_sum_slice)

MRI_array_datasets = np.array(MRI_array_datasets)
MRI_array_datasets = torch.Tensor(MRI_array_datasets[:, np.newaxis, :, :])
# MRI_array_datasets = MRI_array_datasets.expand(-1, 3, -1, -1)


In [ ]:
label = pd.read_csv("./only_tumor_clinical.csv")
MRI_PCR_label = torch.Tensor(label.values[:,5].astype(int))
# MRI_array_datasets = torch.Tensor(MRI_array_datasets).to(device)
# train_dataset = TensorDataset(MRI_array_datasets)


resnet101

In [9]:
from torchvision.models import resnet

batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet.resnet101(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2).to(device)
model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


model.load_state_dict(torch.load('best_resnet50_train_1.pth'))
model = nn.Sequential(*list(model.children())[:-1])

model.to(device)

model_data = TensorDataset(MRI_array_datasets,MRI_PCR_label)
model_dataloader = DataLoader(dataset=model_data,batch_size=batch_size,shuffle=True)
model.eval()

val_result = []
val_label = []
with torch.no_grad():
    for data, label in model_dataloader:
        data = data.to(device)
        label = label.long().to(device)
        feature = model(data)
        val_result.append(feature)
        val_label.append(label)

val_result = torch.cat(val_result).cpu()
val_label = torch.cat(val_label).cpu()
# auc = roc_auc_score(val_label.cpu().numpy(), val_result.detach().cpu().numpy())


c:\Users\HongDQ\.conda\envs\fb\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HongDQ\.conda\envs\fb\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


data_total = pd.DataFrame(np.array(val_result).reshape(929,2048))
# val_label
# labels = label.values[:,5].astype(float)
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()

data_total = (standard_scaler.fit_transform(data_total.T)).T

# 创建LASSO回归模型，使用LassoCV自动选择最佳的alpha（惩罚参数)
lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0])  # 通过交叉验证选择alpha值
lasso.fit(data_total, val_label)

# 打印特征系数
feature_coefficients = dict(zip(range(1, len(lasso.coef_) + 1), lasso.coef_))
best_auc = 0


In [14]:
num_selected_features = 2000   # 根据需要选择特定数量的特征
selected_features = [feature for feature, coefficient in sorted(feature_coefficients.items(), key=lambda x: abs(x[1]), reverse=True)[:num_selected_features]]

# 将数据分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data_total[:,selected_features], val_label, test_size=0.2, random_state=42)

# 创建逻辑回归模型
model = LogisticRegression()

# 在训练集上训练模型
model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = model.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'准确率: {accuracy}')

# 打印分类报告
report = classification_report(y_test, y_pred)
print(report)

y_prob = model.predict_proba(X_test)[:, 1]  # 得到正类别的概率
auc = roc_auc_score(y_test, y_prob)
print(auc)
# if auc > best_auc:
#     best_auc = auc
# print(f'AUC: {auc}')
#     print(best_auc)


准确率: 0.8387096774193549
              precision    recall  f1-score   support

           0       0.89      0.86      0.88       125
           1       0.74      0.79      0.76        61

    accuracy                           0.84       186
   macro avg       0.82      0.83      0.82       186
weighted avg       0.84      0.84      0.84       186

0.9215737704918033


c:\Users\HongDQ\.conda\envs\fb\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
MRI_array_datasets = torch.Tensor(np.array(image_max_slic_total)[:, np.newaxis, :, :])
MRI_array_datasets = MRI_array_datasets.expand(-1, 3, -1, -1)

label = pd.read_csv("./only_tumor_clinical.csv")
MRI_PCR_label = torch.Tensor(label.values[:,5].astype(int))


In [ ]:
from torchvision.models import resnet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 32

model = resnet.resnet101(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2).to(device)

# model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

model.to(device)

model_data = TensorDataset(MRI_array_datasets,MRI_PCR_label)
model_dataloader = DataLoader(dataset=model_data,batch_size=batch_size,shuffle=True)
model.eval()

val_result = []
val_label = []
with torch.no_grad():
    for data, label in model_dataloader:
        data = data.to(device)
        label = label.long().to(device)
        output = model(data)
        x = output[:,1]
        val_result.append(x)
        val_label.append(label)

val_result = torch.cat(val_result).cpu()
val_label = torch.cat(val_label).cpu()
auc = roc_auc_score(val_label.cpu().numpy(), val_result.detach().cpu().numpy())
print(auc)


In [ ]:
from torchvision.models import resnet
from torch.optim.lr_scheduler import CosineAnnealingLR

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet.resnet101(pretrained=True)
# model.load_state_dict(torch.load('resnet101-63fe2227.pth'))

# model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

model.fc = nn.Linear(model.fc.in_features, 2).to(device)
model.to(device)

model.eval()

batch_size = 32
learning_rate = 0.00001
num_epoch = 100
Loss_MRI = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)
scheduler = CosineAnnealingLR(optimizer, T_max=num_epoch)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(MRI_array_datasets, MRI_PCR_label):
    X_train, X_test = MRI_array_datasets[train_index], MRI_array_datasets[test_index]
    y_train, y_test = MRI_PCR_label[train_index], MRI_PCR_label[test_index]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2,random_state=42)
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)
    train_DataLoader = DataLoader(dataset = train_dataset,batch_size=batch_size, shuffle=True)
    val_DataLoader = DataLoader(dataset= val_dataset,batch_size= batch_size, shuffle=True)
    test_DataLoader = DataLoader(dataset = test_dataset,batch_size= batch_size, shuffle=True)
    
    loss_total = []
    patience = 5
    current_patience = 0

    train_auc = []

    val_auc = []
    best_val_auc = 0.0

    for epoch in range(num_epoch):
        model.train()
        scheduler.step()
        loss_epoch = 0
        train_result = []
        train_label = []

        for data, label in train_DataLoader:
            data = data.to(device)
            label = label.long().to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = Loss_MRI(output, label)
            loss_epoch = loss_epoch + loss
            loss.backward()
            optimizer.step()
            x = output[:,1]
            train_result.append(x)
            train_label.append(label)

        loss_total.append(loss_epoch)
        train_result = torch.cat(train_result).cpu()
        train_label = torch.cat(train_label).cpu()
        auc = roc_auc_score(train_label.cpu().numpy(), train_result.detach().cpu().numpy())
        train_auc.append(auc)

        print(f'Epoch [{epoch + 1}/{num_epoch}], Loss: {loss.item():.4f}')

    # if (epoch+1)%10 == 0:
        model.eval()
        with torch.no_grad():
            val_result = []
            val_label = []
            for data, label in val_DataLoader:
                data = data.to(device)
                label = label.long().to(device)
                output = model(data)
                x = output[:,1]
                val_result.append(x)
                val_label.append(label)
            
            val_result = torch.cat(val_result).cpu()
            val_label = torch.cat(val_label).cpu()
            auc = roc_auc_score(val_label.cpu().numpy(), val_result.detach().cpu().numpy())
            val_auc.append(auc)
            if auc > best_val_auc:
                current_patience = 0
                best_val_auc = auc
                torch.save(model.state_dict(), 'best_resnet50_train_1.pth')
                print(f'classify')
                print(best_val_auc)
            else:
                current_patience += 1
        if current_patience >= patience:
            print(f'###################Early stopping###################')
            break
    
            
    # loss_total = torch.tensor(loss_total).cpu().numpy()

    # plt.plot(range(num_epoch), loss_total)
    # plt.xlabel('Epochs')
    # plt.ylabel('Loss')
    # plt.title('Training Loss Curve')
    # plt.savefig('train_loss.png')

    # plt.plot(range(num_epoch), train_auc)
    # plt.xlabel('Epochs')
    # plt.ylabel('Training Accuracy')
    # plt.title('Training Accuracy Curve')
    # plt.savefig('train_acc.png')

    # plt.plot(range(num_epoch), val_auc)
    # plt.xlabel('Epochs')
    # plt.ylabel('Validation Accuracy')
    # plt.title('Validation Accuracy Curve')
    # plt.savefig('validation_acc.png')
    
    model.load_state_dict(torch.load('best_resnet50_train_1.pth'))
    model.eval()
    with torch.no_grad():
        test_result = []
        test_label = []
        for data, label in test_DataLoader:
            data = data.to(device)
            label = label.long().to(device)
            output = model(data)
            x = output[:,1]
            test_result.append(x)
            test_label.append(label)
    
        test_result = torch.cat(test_result).cpu()
        test_label = torch.cat(test_label).cpu()
        auc = roc_auc_score(test_label.cpu().numpy(), test_result.detach().cpu().numpy())
        print(auc)
    1/0


In [ ]:
class ConvAutoencoder3D(nn.Module):
    def __init__(self):
        super(ConvAutoencoder3D, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 16, 3, stride=2, padding=1), # input is (1, 60, 60, 60), output is (16, 30, 30, 30)
            nn.AvgPool3d(kernel_size=2,stride=2),
            nn.ReLU(),
            nn.Conv3d(16, 32, 3, stride=2, padding=1), # output is (32, 15, 15, 15)
            nn.AvgPool3d(kernel_size=2,stride=2),
            nn.ReLU(),
            nn.Conv3d(32, 64, 3, stride=2, padding=1), # output is (64, 8, 8, 8)
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(64, 32, 3, stride=2, padding=1, output_padding=1), # output is (32, 15, 15, 15)
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.ConvTranspose3d(32, 16, 3, stride=2, padding=1, output_padding=1), # output is (16, 30, 30, 30)
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.ConvTranspose3d(16, 1, 3, stride=2, padding=1, output_padding=1), # output is (1, 60, 60, 60)
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        feature = x.view(x.size(0),-1)
        x = self.decoder(x)
        return x,feature


In [ ]:
batch_size = 16
learning_rate = 0.001
num_epoch = 1000
Loss_MRI = nn.MSELoss()
model = ConvAutoencoder3D()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(MRI_array_datasets, MRI_PCR_label):
    X_train, X_test = MRI_array_datasets[train_index], MRI_array_datasets[test_index]
    y_train, y_test = MRI_PCR_label[train_index], MRI_PCR_label[test_index]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2,random_state=42)
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)
    train_DataLoader = DataLoader(dataset = train_dataset,batch_size=batch_size, shuffle=True)
    val_DataLoader = DataLoader(dataset= val_dataset,batch_size= batch_size, shuffle=True)
    test_DataLoader = DataLoader(dataset = test_dataset,batch_size= batch_size, shuffle=True)
       
    patience = 5
    current_patience = 0
    best_loss = float('inf')
    loss_total = []


    for epoch in range(num_epoch):
        model.train()
        loss_epoch = 0.0
        data_total = []
        output_total = []

        for data, label in train_DataLoader:
            data = data.to(device)
            data_total.append(data)
            optimizer.zero_grad()
            output,feature = model(data)
            output_total.append(output)
            loss = Loss_MRI(output, data)
            loss_epoch = loss_epoch + loss
            loss.backward()
            optimizer.step()

        data_total = torch.cat(data_total)
        output_total = torch.cat(output_total)
        train_loss = Loss_MRI(output_total,data_total)
        # loss_total.append((train_loss))
        print(f'Epoch [{epoch + 1}/{num_epoch}], Loss: {train_loss.item():.4f}')

        model.eval()
        with torch.no_grad():
            loss_val = 0.0
            data_total = []
            output_total = []
            for data, label in val_DataLoader:
                data = data.to(device)
                output,feature = model(data)
                data_total.append(data)
                output_total.append(output)
            
            data_total = torch.cat(data_total)
            output_total = torch.cat(output_total)
            val_loss = Loss_MRI(output_total,data_total)
            
            if val_loss < best_loss:
                # current_patience = 0
                best_loss = val_loss
                torch.save(model.state_dict(), 'best_AE_MRI_train.pth')
                print(f'AE')
                print(best_loss)
        #     else:
        #         current_patience += 1
        # if current_patience >= patience:
        #     print(f'###################Early stopping###################')
        #     break
    
    # plt.plot(range(num_epoch), loss_total)
    # plt.xlabel('Epochs')
    # plt.ylabel('Loss')
    # plt.title('Training Loss Curve')
    # plt.savefig('train_loss.png')

    # plt.plot(range(num_epoch), train_auc)
    # plt.xlabel('Epochs')
    # plt.ylabel('Training Accuracy')
    # plt.title('Training Accuracy Curve')
    # plt.savefig('train_acc.png')

    # plt.plot(range(num_epoch), val_auc)
    # plt.xlabel('Epochs')
    # plt.ylabel('Validation Accuracy')
    # plt.title('Validation Accuracy Curve')
    # plt.savefig('validation_acc.png')
    

    model.load_state_dict(torch.load('best_AE_MRI_train.pth'))
    model.eval()
    with torch.no_grad():
        loss_test = 0.0
        data_total = []
        output_total = []
        feature_total = []
        for data, label in test_DataLoader:
            data = data.to(device)
            label = label.long().to(device)
            output,feature = model(data)
            feature_total.append(feature)
            data_total.append(data)
            output_total.append(output)

        data_total = torch.cat(data_total)
        output_total = torch.cat(output_total)
        test_loss = Loss_MRI(output_total,data_total)
        # test_result = torch.cat(test_result).cpu()
        # test_label = torch.cat(test_label).cpu()
        # auc = roc_auc_score(test_label.cpu().numpy(), test_result.detach().cpu().numpy())
        print(test_loss)

    1/0

In [ ]:
batch_size = 32
learning_rate = 0.000000001
num_epoch = 100
Loss_MRI = nn.CrossEntropyLoss()
model = monai.networks.nets.resnet.ResNet(block=Bottleneck, layers=(3, 4, 6, 3), block_inplanes=(64, 128, 256, 512), n_input_channels=1, num_classes=2)
model.to(device)
# parme_path = "./fmcib.torch"
# checkpoint = torch.load(parme_path, map_location=device)
# model.load_state_dict(checkpoint, strict=False)

optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(MRI_array_datasets, MRI_PCR_label):
    X_train, X_test = MRI_array_datasets[train_index], MRI_array_datasets[test_index]
    y_train, y_test = MRI_PCR_label[train_index], MRI_PCR_label[test_index]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2,random_state=42)
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)
    train_DataLoader = DataLoader(dataset = train_dataset,batch_size=batch_size, shuffle=True)
    val_DataLoader = DataLoader(dataset= val_dataset,batch_size= batch_size, shuffle=True)
    test_DataLoader = DataLoader(dataset = test_dataset,batch_size= batch_size, shuffle=True)
    
    loss_total = []
    patience = 5
    current_patience = 0

    train_auc = []

    val_auc = []
    best_val_auc = 0.0

    for epoch in range(num_epoch):
        model.train()
        loss_epoch = 0
        train_result = []
        train_label = []

        for data, label in train_DataLoader:
            data = data.to(device)
            label = label.long().to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = Loss_MRI(output, label)
            loss_epoch = loss_epoch + loss
            loss.backward()
            optimizer.step()
            x = output[:,1]
            train_result.append(x)
            train_label.append(label)

        loss_total.append(loss_epoch)
        train_result = torch.cat(train_result).cpu()
        train_label = torch.cat(train_label).cpu()
        auc = roc_auc_score(train_label.cpu().numpy(), train_result.detach().cpu().numpy())
        train_auc.append(auc)

        print(f'Epoch [{epoch + 1}/{num_epoch}], Loss: {loss.item():.4f}')

    # if (epoch+1)%10 == 0:
        model.eval()
        with torch.no_grad():
            val_result = []
            val_label = []
            for data, label in val_DataLoader:
                data = data.to(device)
                label = label.long().to(device)
                output = model(data)
                x = output[:,1]
                val_result.append(x)
                val_label.append(label)
            
            val_result = torch.cat(val_result).cpu()
            val_label = torch.cat(val_label).cpu()
            auc = roc_auc_score(val_label.cpu().numpy(), val_result.detach().cpu().numpy())
            val_auc.append(auc)
            if auc > best_val_auc:
                # current_patience = 0
                best_val_auc = auc
                torch.save(model.state_dict(), 'best_monai_3D-resnet_train.pth')
                print(f'classify')
                print(best_val_auc)
        #     else:
        #         current_patience += 1
        # if current_patience >= patience:
        #     print(f'###################Early stopping###################')
        #     break
    
            
    loss_total = torch.tensor(loss_total).cpu().numpy()

    plt.plot(range(num_epoch), loss_total)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.savefig('train_loss.png')

    plt.plot(range(num_epoch), train_auc)
    plt.xlabel('Epochs')
    plt.ylabel('Training Accuracy')
    plt.title('Training Accuracy Curve')
    plt.savefig('train_acc.png')

    plt.plot(range(num_epoch), val_auc)
    plt.xlabel('Epochs')
    plt.ylabel('Validation Accuracy')
    plt.title('Validation Accuracy Curve')
    plt.savefig('validation_acc.png')
    

    model.load_state_dict(torch.load('best_monai_3D-resnet_train.pth'))
    model.eval()
    with torch.no_grad():
        test_result = []
        test_label = []
        for data, label in test_DataLoader:
            data = data.to(device)
            label = label.long().to(device)
            output = model(data)
            x = output[:,1]
            test_result.append(x)
            test_label.append(label)
          
        test_result = torch.cat(test_result).cpu()
        test_label = torch.cat(test_label).cpu()
        auc = roc_auc_score(test_label.cpu().numpy(), test_result.detach().cpu().numpy())
        print(auc)

    1/0

In [ ]:
batch_size = 32

Loss_MRI = nn.CrossEntropyLoss()
model = monai.networks.nets.resnet.ResNet(block=Bottleneck, layers=(3, 4, 6, 3), block_inplanes=(64, 128, 256, 512), n_input_channels=1, num_classes=2)
model.to(device)
parme_path = "./fmcib.torch"
checkpoint = torch.load(parme_path, map_location=device)
model.load_state_dict(checkpoint, strict=False)


model_data = TensorDataset(MRI_array_datasets,MRI_PCR_label)
model_dataloader = DataLoader(dataset=model_data,batch_size=batch_size,shuffle=True)
model.eval()

val_result = []
val_label = []
with torch.no_grad():
    for data, label in model_dataloader:
        data = data.to(device)
        label = label.long().to(device)
        output = model(data)
        x = output[:,1]
        val_result.append(x)
        val_label.append(label)

val_result = torch.cat(val_result).cpu()
val_label = torch.cat(val_label).cpu()
auc = roc_auc_score(val_label.cpu().numpy(), val_result.detach().cpu().numpy())
